<a href="https://colab.research.google.com/github/your-lithium/projector-project/blob/main/notebooks/scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## imports

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
import logging
logging.getLogger('urllib3').setLevel(logging.ERROR)

In [ ]:
import time

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import re

In [ ]:
import ast

## scraping

In [ ]:
def scrape_cars():
    url = 'https://auto.ria.com/uk/car/used/'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

    cars_list = []

    for i in range(3400):
        start = time.time()
        response = requests.get((url + '?page=' + str(i+1)), headers=headers)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            car_elements = soup.find_all('section', class_='ticket-item')

            for car in car_elements:
                car_info = {}

                div = car.find('div', class_='hide')
                car_info['equipment'] = div.get('data-equipment-name')
                car_info['generation'] = div.get('data-generation-name')
                car_info['mark'] = div.get('data-mark-name')
                car_info['model'] = div.get('data-model-name')
                car_info['modification'] = div.get('data-modification-name')
                car_info['year'] = div.get('data-year')

                div = car.find('div', class_='content-bar')
                car_info['link'] = div.find('a').get('href')

                div = div.find('div', class_='content')

                div_price = div.find('div', class_='price-ticket')
                car_info['currency'] = div_price.get('data-main-currency')
                car_info['price'] = div_price.get('data-main-price')

                div = div.find('div', class_='definition-data')

                div_details = div.find('ul', class_='unstyle characteristic')
                div_details = div_details.find_all('li')
                car_info['mileage'] = div_details[0].get_text()
                car_info['place'] = div_details[1].get_text()
                car_info['gearbox'] = div_details[3].get_text()

                additional_info = {}
                div_info = div.find('div', class_='base_information')
                if div_info:
                    additional_info['number'] = bool(div_info.find(class_='state-num ua'))
                    additional_info['vin'] = bool(div_info.find(class_='label-vin'))
                    additional_info['vin_checked'] = bool(div_info.find(class_='popup __larr financeua'))

                    details = div_info.find_all(class_='item')
                    if details:
                        additional_info['details'] = [detail.get_text() for detail in details]

                    car_info['additional_info'] = additional_info
                else:
                    car_info['additional_info'] = None

                div_element = div.find(class_='descriptions-ticket')
                car_info['description'] = div_element.find('span').get_text()

                # continue scraping in separate car page
                # car_response = requests.get(link, headers=headers)
                # soup = BeautifulSoup(car_response.content, 'html.parser')

                # info = soup.find('div', 'technical-info')
                # print(info)

                # car_info['engine_type'] = soup.
                # car_info['engine_capacity'] =

                cars_list.append(car_info)
        end = time.time()
        print(end - start)

    return cars_list

cars = scrape_cars()

In [ ]:
cars

In [ ]:
data = pd.DataFrame(cars)

In [ ]:
data

,equipment,generation,mark,model,modification,year,link,currency,price,mileage,place,gearbox,additional_info,description
0,Base,IV покоління/J (FL),Opel,Astra,1.6 АТ (115 к.с.),2016,https://auto.ria.com/uk/auto_opel_astra_364925...,USD,13400,50 тис. км,Одеса ( від ),Автомат,"{'number': True, 'vin': True, 'vin_checked': T...",EXLUSIVE!!! НОВАЯ МАШИНА!!! 49 Т.КМ ОРИГИНАЛЬН...
1,,E70 (FL),BMW,X5,,2013,https://auto.ria.com/uk/auto_bmw_x5_36532333.html,USD,22000,181 тис. км,Кривий Ріг ( від ),Автомат,"{'number': True, 'vin': True, 'vin_checked': T...",Продам ОФИЦИАЛЬНЫЙ Х 5\nОригинальный пробег\nА...
2,Base,I покоління,Hyundai,i40,2.0 AT (166 к.с.),2012,https://auto.ria.com/uk/auto_hyundai_i40_36530...,USD,16500,19 тис. км,Одеса ( від ),Автомат,"{'number': True, 'vin': True, 'vin_checked': T...",ЕКСКЛЮЗИВНЕ ОФІЦІЙНЕ АВТО!!!Машина нова!!! 190...
3,M Package,F15,BMW,X5,M50d Steptronic (381 к.с.) xDrive,2014,https://auto.ria.com/uk/auto_bmw_x5_36302146.html,USD,33999,250 тис. км,Одеса ( від ),Автомат,"{'number': True, 'vin': True, 'vin_checked': T...",Обмін бажано на G-05 з моею доплатою .Продаемо...
4,Base,G20,BMW,3 Series,320d Steptronic (190 к.с.) xDrive,2019,https://auto.ria.com/uk/auto_bmw_3_series_3646...,USD,33500,142 тис. км,Одеса ( від ),Автомат,"{'number': True, 'vin': True, 'vin_checked': T...",Ціна за налічку без торгу!!!!ОФІЦІЙНИЙ БАВАРЕЦ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67993,,III покоління (FL),Skoda,Superb,2.0 TSI 7DSG (190 к.с.),2023,https://auto.ria.com/uk/newauto/auto-skoda-sup...,USD,35748,без пробігу,Рівне ( від ),Роботизована,"{'number': False, 'vin': False, 'vin_checked':...",\n
67994,Business+,IV покоління/L10 (FL),Lexus,GS,200t AT (245 к.с.),2016,https://auto.ria.com/uk/auto_lexus_gs_36326499...,USD,23490,120 тис. км,Львів ( від ),Автомат,"{'number': False, 'vin': True, 'vin_checked': ...",Код товару: 42826.\n\nМожливий обмін на будь-я...
67995,,,Opel,Astra,,1999,https://auto.ria.com/uk/auto_opel_astra_360417...,USD,3700,318 тис. км,Миколаїв ( від ),Ручна / Механіка,"{'number': True, 'vin': True, 'vin_checked': T...",Авто в доброму стані. Мала витрата пального. Н...
67996,Bose,III покоління (2nd FL),Renault,Scenic,1.6 dCi MT (130 к.с.) Start&Stop,2013,https://auto.ria.com/uk/auto_renault_scenic_36...,USD,9550,175 тис. км,Київ ( від ),Ручна / Механіка,"{'number': False, 'vin': False, 'vin_checked':...",Продам Renault Scenic 1.6 DCI BOSE\nМаксимальн...


In [ ]:
data.to_csv('/content/cars.csv')

## beautifying

In [ ]:
data = pd.read_csv('/content/cars.csv', index_col=0)

In [ ]:
cols = data.columns.tolist()
cols

['Unnamed: 0',
 'equipment',
 'generation',
 'mark',
 'model',
 'modification',
 'year',
 'link',
 'currency',
 'price',
 'mileage',
 'place',
 'gearbox',
 'additional_info',
 'description']

In [ ]:
cols = ['currency', 'price', 'link', 'mark', 'model', 'year', 'modification',
        'equipment', 'generation', 'mileage', 'place', 'gearbox',
        'additional_info', 'description']

data = data[cols]

In [ ]:
def clean_place(place):
    pattern = '(.+) \( від \)'
    matched = re.search(pattern, place)

    if matched:
      return matched.group(1)
    else:
      return None

data["place"] = data["place"].apply(lambda x: clean_place(x))

In [ ]:
def clean_mileage(mileage):
    pattern = '(\d+) тис\. км'
    matched = re.search(pattern, mileage)

    if matched:
        return int(matched.group(1)) * 1000
    else:
        pattern = ' без пробігу '
        matched = re.search(pattern, mileage)
        if matched:
            return 0
        else:
            print(mileage)
            return None

data["mileage"] = data["mileage"].apply(lambda x: clean_mileage(x))

In [ ]:
loan_list = ['Кредит до 1 року під 0.01%',
            'Кредит до 2 років під 0.01%',
            'Кредит до 3 років під 0.01%',
            'Кредит до 4 років під 0.01%',
            'Кредит до 5 років під 0.01%',
            'Кредит до 5 років під 14.9%']

In [ ]:
def decipher_additional_info(additional_info):
    additional_info = ast.literal_eval(additional_info)

    number = additional_info['number']
    vin = additional_info['vin']
    vin_checked = additional_info['vin_checked']

    try:
        details = additional_info['details']

        if ' Акція' in details or 'Акція' in details:
            promo = True
        else:
            promo = False

        if ' Взято в кредит' in details or 'Взято в кредит' in details:
            on_credit = True
        else:
            on_credit = False

        if '7 років гарантії' in details:
            warranty = True
        else:
            warranty = False

        if 'Без першої реєстрації' in details:
            no_registration = True
        else:
            no_registration = False

        if 'Був в ДТП' in details:
            accident = True
        else:
            accident = False

        if 'Доступно в лізинг' in details:
            lease_available = True
        else:
            lease_available = False

        if 'Зафіксований курс' in details:
            fixed_rate = True
        else:
            fixed_rate = False

        if 'Краща ціна' in details:
            best_price = True
        else:
            best_price = False

        if 'Не на ходу' in details:
            in_running_order = False
        else:
            in_running_order = True

        if 'Онлайн бронювання' in details:
            online_booking = True
        else:
            online_booking = False

        if 'Остання пропозиція' in details:
            last_offer = True
        else:
            last_offer = False

        if 'Продає AUTO.RIA' in details:
            seller_is_auto_ria = True
        else:
            seller_is_auto_ria = False

        loan = next((element for element in loan_list if element in details), None)
        if loan:
            pattern = 'Кредит до (\d) рок(?:у|ів) під (\d{1,2}\.\d{1,2})%'
            matched = re.search(pattern, loan)
            loan_years = int(matched.group(1))
            loan_rate = float(matched.group(2))
        else:
            loan_years = None
            loan_rate = None

    except KeyError:
        promo = False
        on_credit = False
        warranty = False
        no_registration = False
        accident = False
        lease_available = False
        fixed_rate = False
        best_price = False
        in_running_order = True
        online_booking = False
        last_offer = False
        seller_is_auto_ria = False
        loan_years = None
        loan_rate = None

    return number, vin, vin_checked, promo, on_credit, warranty, no_registration, accident, lease_available, fixed_rate, best_price, in_running_order, online_booking, last_offer, seller_is_auto_ria, loan_years, loan_rate

In [ ]:
new_columns = ['number', 'vin', 'vin_checked', 'promo', 'on_credit', 'warranty', 'no_registration', 'accident', 'lease_available', 'fixed_rate', 'best_price', 'in_running_order', 'online_booking', 'last_offer', 'seller_is_auto_ria', 'loan_years', 'loan_rate']
new_series = data['additional_info'].apply(lambda x: decipher_additional_info(x))

new_data = pd.DataFrame(new_series.tolist(), columns=new_columns)
data = pd.concat([data, new_data], axis=1)
data.drop(columns=['additional_info'], inplace=True)

In [ ]:
data

,currency,price,link,mark,model,year,modification,equipment,generation,mileage,...,accident,lease_available,fixed_rate,best_price,in_running_order,online_booking,last_offer,seller_is_auto_ria,loan_years,loan_rate
0,USD,13400,https://auto.ria.com/uk/auto_opel_astra_364925...,Opel,Astra,2016,1.6 АТ (115 к.с.),Base,IV покоління/J (FL),50000,...,False,False,False,False,True,False,False,False,NaN,NaN
1,USD,22000,https://auto.ria.com/uk/auto_bmw_x5_36532333.html,BMW,X5,2013,NaN,NaN,E70 (FL),181000,...,False,False,False,False,True,False,False,False,NaN,NaN
2,USD,16500,https://auto.ria.com/uk/auto_hyundai_i40_36530...,Hyundai,i40,2012,2.0 AT (166 к.с.),Base,I покоління,19000,...,False,False,False,False,True,False,False,False,NaN,NaN
3,USD,33999,https://auto.ria.com/uk/auto_bmw_x5_36302146.html,BMW,X5,2014,M50d Steptronic (381 к.с.) xDrive,M Package,F15,250000,...,True,False,False,False,True,False,False,False,NaN,NaN
4,USD,33500,https://auto.ria.com/uk/auto_bmw_3_series_3646...,BMW,3 Series,2019,320d Steptronic (190 к.с.) xDrive,Base,G20,142000,...,False,False,False,False,True,False,False,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67993,USD,35748,https://auto.ria.com/uk/newauto/auto-skoda-sup...,Skoda,Superb,2023,2.0 TSI 7DSG (190 к.с.),NaN,III покоління (FL),0,...,False,True,False,False,True,False,False,False,3.0,0.01
67994,USD,23490,https://auto.ria.com/uk/auto_lexus_gs_36326499...,Lexus,GS,2016,200t AT (245 к.с.),Business+,IV покоління/L10 (FL),120000,...,False,False,False,False,True,False,False,False,NaN,NaN
67995,USD,3700,https://auto.ria.com/uk/auto_opel_astra_360417...,Opel,Astra,1999,NaN,NaN,NaN,318000,...,False,False,False,False,True,False,False,False,NaN,NaN
67996,USD,9550,https://auto.ria.com/uk/auto_renault_scenic_36...,Renault,Scenic,2013,1.6 dCi MT (130 к.с.) Start&Stop,Bose,III покоління (2nd FL),175000,...,False,False,False,False,True,False,False,False,NaN,NaN


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67998 entries, 0 to 67997
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   currency            67998 non-null  object 
 1   price               67998 non-null  int64  
 2   link                67998 non-null  object 
 3   mark                67996 non-null  object 
 4   model               67990 non-null  object 
 5   year                67998 non-null  int64  
 6   modification        35685 non-null  object 
 7   equipment           24071 non-null  object 
 8   generation          49833 non-null  object 
 9   mileage             67998 non-null  int64  
 10  place               67998 non-null  object 
 11  gearbox             67998 non-null  object 
 12  description         62765 non-null  object 
 13  number              67998 non-null  bool   
 14  vin                 67998 non-null  bool   
 15  vin_checked         67998 non-null  bool   
 16  prom

In [ ]:
data.to_csv('/content/cars.csv')